In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas_datareader as pdr
import pandas as pd
import statsmodels.api as sm
import seaborn as sns
import sklearn as skl
import tensorflow as tf
import seaborn as sns
import sklearn as skl
from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.techindicators import TechIndicators

In [2]:
crypto = ['DASH-USD','EOS-USD','LTC-USD','BNB-USD','TRX-USD','XLM-USD','BCH-USD', 'DASH-USD']
commodities = ['ES=F', 'CL=F', 'GC=F', 'NG=F', 'ZN=F', 'NQ=F', 'C=F', 'S=F']

In [3]:
api_key = "QO234RMNT7YYSXZ3"
period = 60
ts = TimeSeries(key=api_key, output_format="pandas")
allrawdfs = []
def extract_df(data):
    for i in data:
        allrawdfs.append(ts.get_intraday(i.upper(), interval="5min", outputsize="full")[0])

In [4]:
extract_df(crypto[ : 5])

In [7]:
extract_df(crypto[ 4 : ])

In [8]:
extract_df(commodities[ : 5])

In [11]:
extract_df(commodities[ 4 : ])

In [12]:
alldfs = []
def extract_vols(data):
    mean_volumes = []
    for i in data:
        mean_volumes.append(i['5. volume'].mean())
    vol_array = np.array(mean_volumes)
    for i in data:
        if i['5. volume'].mean() > np.quantile(vol_array, 0.5):
            alldfs.append(i)

In [13]:
extract_vols(allrawdfs)

In [14]:
dfs_with_diffs = []
def add_diffs(data):
    for i in data:
        i['diff'] = i['4. close'] - i['1. open']
        dfs_with_diffs.append(i)

In [15]:
add_diffs(alldfs)

In [16]:
allPrice = []
def getclose(data):
    for i in data:
        new  = list(i['4. close'].to_numpy())
        new.reverse()
        allPrice.append(new)

In [17]:
getclose(dfs_with_diffs)

In [18]:
final = []
def get_support_resistance(data):
    result = []
    all_data = []
    counts = []
    for x in data:
        all_data.append( [ x, data.count(x)] )
        counts.append(data.count(x))
    unrepeated = []
    for x in all_data:
        if x not in unrepeated:
            unrepeated.append(x)
    sorted_result = sorted(unrepeated, key = lambda i: i[1], reverse=True)
    arr = np.array(sorted_result)

    for x in list(arr):
        if x[1] > np.quantile(np.array(counts), 0.78):
            result.append(list(x))
    return result
def get_s_r(data):
    for i in data:
        final.append(get_support_resistance(i))

In [19]:
get_s_r(allPrice)

In [20]:
converted_dfs = []
def covert_to_list(data):
    for i in data:
        new = i.values.tolist()
        converted_dfs.append(new)

In [21]:
covert_to_list(dfs_with_diffs)

In [62]:
extracted_points = []
extracted_inputs = []
extracted_outputs = []
def get_spread(data, price):
    inputs = []
    outputs = []
    res = []
    for x in data:  
        for i in price:
        # by doing this, we are able to extract these points from the entire list and get those with a good spread
            if x[3] == i[0]:
                diff = x[0] - x[3]
                inputs.append(list(data[ data.index(x) - 5 : data.index(x)]))
                outputs.append(list(data[ data.index(x) : data.index(x) + 8 ]))
                res.append(x)
    return [res, inputs, outputs]
def get_s(data, price):
    for a,b in zip(data, price):
        extracted_points.append(get_spread(a,b)[0])
        extracted_inputs.append(get_spread(a,b)[1])
        extracted_outputs.append(get_spread(a,b)[2])

In [63]:
get_s(converted_dfs,final)

In [64]:
diff_on_prev = []
def get_ins(data):
    newDiff =[]
    for x in data:
        def extract(v):
            return v[5]
        result = map(extract, x)
        newDiff.append(list(result))
    return newDiff
def get_i(data):
    for i in data:
        diff_on_prev.append(get_ins(i))

In [65]:
get_i(extracted_inputs)

In [66]:
sum_diff_on_next = []
def get_outs(data):
    rightD = []
    summation = []
    for r in data:
        def extractr(f):
            return f[5]
        resultr = map(extractr, r)
        rightD.append(list(resultr))
    for p in rightD:
        summation.append(sum(p))
    return summation
def get_o(data):
    for i in data:
        sum_diff_on_next.append(get_outs(i))

In [67]:
get_o(extracted_outputs)

In [68]:
 transact_or_not = []
 def get_decision(data):
    sum_np_array = np.array(data)
    decision = []
    for t in data:
        if abs(t) > np.quantile(sum_np_array, 0.99):
        # the 1 stands for a transaction while the 0 stands for no transaction
            decision.append(1)
        else:
            decision.append(0)
    return decision
def get_d(data):
    for i in data:
        transact_or_not.append(get_decision(i))

In [69]:
get_d(sum_diff_on_next)

In [70]:
# We figured out that these only provide us with less transactions and has small breakouts. We have to get much more data that's volatile and extract better equilibriums with better breakouts. Run different apis after every minute, increase the outputs, figure out a relation with SMA

In [71]:
np.array(sum_diff_on_next[7]).max()

2.3703000000000145

In [72]:
t_or_n = []
combinedinputs = []
combinedoutputs = []
for a,b,c in zip(diff_on_prev,sum_diff_on_next,transact_or_not):
    combinedinputs.extend(a)
    combinedoutputs.extend(b)
    t_or_n.extend(c)

In [73]:
headers = ['I1','I2','I3','I4','I5']
df = pd.DataFrame(combinedinputs)
df.columns = headers
df['Transaction'] = t_or_n
df['Outputs'] = combinedoutputs
df.describe()

,I1,I2,I3,I4,I5,Transaction,Outputs
count,5356.000000,5356.000000,5356.000000,5356.000000,5356.000000,5358.000000,5358.000000
mean,0.018059,0.009134,0.008989,0.003217,-0.015195,0.155842,0.068179
std,0.954822,0.952051,0.977665,1.004808,0.985326,0.362739,2.599773
min,-13.750000,-17.000000,-17.000000,-13.750000,-13.750000,0.000000,-20.750000
25%,-0.001500,-0.001600,-0.001300,-0.001400,-0.001700,0.000000,-0.006100
50%,0.000000,0.000000,0.000000,0.000000,0.000100,0.000000,-0.000100
75%,0.001000,0.000700,0.001200,0.001025,0.000700,0.000000,0.001775
max,12.250000,14.250000,14.250000,14.250000,14.250000,1.000000,49.250000


In [81]:
unscaled_inputs = df.iloc[ :, 0 : 5 ]
targets_all = df.iloc[ :, -2]

In [108]:
num_one_targets = int(np.sum(targets_all))
zero_counter = 0
indices_to_remove = []
for i in range(targets_all.shape[0]):
    if targets_all[i] == 0:
        zero_counter += 1
        if zero_counter > num_one_targets:
            indices_to_remove.append(i)
unscaled_inputs_equal_priors = np.delete(unscaled_inputs.values.tolist(), indices_to_remove, axis = 0)            
targets_equal_priors = np.delete(targets_all.values.tolist(), indices_to_remove, axis = 0)            

In [111]:
from sklearn import preprocessing
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)

In [112]:
shuffled_indices  = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

In [113]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

In [116]:
x_train, x_test, y_train, y_test = train_test_split(shuffled_inputs, shuffled_targets, random_state=0, test_size = 0.2)

In [118]:
import math
math.sqrt(len(shuffled_targets))

40.8656334834051

In [119]:
classifier = KNeighborsClassifier(n_neighbors=39, p=2, metric = 'euclidean')
classifier.fit(x_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='euclidean',
                     metric_params=None, n_jobs=None, n_neighbors=39, p=2,
                     weights='uniform')

In [120]:
y_pred = classifier.predict(x_test)
y_pred

array([1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0,
       1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1,
       0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1,

In [121]:
cm = confusion_matrix(y_test,y_pred)
print(cm)

[[158  17]
 [ 16 143]]


In [122]:
f1_score(y_test,y_pred)

0.896551724137931

In [123]:
print(accuracy_score(y_test,y_pred))

0.9011976047904192
